<h1>Sentimentanalyse der Twitter Daten</h1>

<h2>Aufgaben</h2>
<p>Laden der Daten - DONE -</p>
<p>Wie kann ich VADER nutzen um eine Sentimentanalyse durchzuführen</p>
<p>Zusammenführen der Daten</p>
<p>Aussortieren nicht benötigter Tweets und Daten</p>


In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
#If not downloaded:
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import flair
import datetime
import stanza
from germansentiment import SentimentModel

<h2>Sentiment Test</h2>

<h3>VADER</h3>

<p>Der versuche eine einfache Sentimentanalyse mit VADER anhand von Beispieldaten durchzuführen</p>

In [2]:
sid = SentimentIntensityAnalyzer()

In [3]:
# Define a list of product reviews

product_reviews = [
    'I love this product. It helps me get so much work done. I tell everyone about what a great thing it is.',
    'This product is defective. I feel like it is broken because it does not do what it promises. Do not buy this.',
    'Do yourself a favor and buy this product as soon as possible. I recommend it to everyone I know. It has saved me so much time!',
    'This product is overpriced and useless. It was a waste of money and it made all my hair fall out.',
    'Works like a dream and it is a bargain! It solves my problems with ease. I bought two!',
    'Do not buy! This product is a ripoff. I wish it was better, but it fails constantly. What a mistake!',
    'This thing is garbage. Do yourself a favor and save the money. Mine is a dumpster fire and fell apart.',
    'I adore this product. =) It makes my life so much easier. And it is a deal!'
]

In [4]:
# For each review in our `product_reviews` list
# Store a polarity score in `scores`
# Then print the score followed by the review
for review in product_reviews:
    scores = sid.polarity_scores(review)
    print(scores['compound'], review)

0.8979 I love this product. It helps me get so much work done. I tell everyone about what a great thing it is.
-0.2263 This product is defective. I feel like it is broken because it does not do what it promises. Do not buy this.
0.807 Do yourself a favor and buy this product as soon as possible. I recommend it to everyone I know. It has saved me so much time!
-0.6808 This product is overpriced and useless. It was a waste of money and it made all my hair fall out.
0.7772 Works like a dream and it is a bargain! It solves my problems with ease. I bought two!
-0.6792 Do not buy! This product is a ripoff. I wish it was better, but it fails constantly. What a mistake!
0.4404 This thing is garbage. Do yourself a favor and save the money. Mine is a dumpster fire and fell apart.
0.8799 I adore this product. =) It makes my life so much easier. And it is a deal!


In [5]:
text = "This thing is garbage Do yourself a favor and save the money Mine is a dumpster fire and fell apart"
for each in text.split():
    if each in sid.lexicon:
        print(each, sid.lexicon[each])
sid.polarity_scores(text)

favor 1.7
save 2.2
dumpster -0.6
fire -1.4


{'neg': 0.167, 'neu': 0.586, 'pos': 0.247, 'compound': 0.4404}

In [6]:
print(sid.polarity_scores("This is good"))
print(sid.polarity_scores("This is Good"))
print(sid.polarity_scores("This is GOOD"))

{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'compound': 0.4404}
{'neg': 0.0, 'neu': 0.355, 'pos': 0.645, 'compound': 0.5622}


In [7]:
like = "I like this"
dont_like = "I do not like this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'compound': 0.3612}
{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'compound': -0.2755}


In [ ]:
like = "I like this!"
dont_like = "I do not like this!"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

In [ ]:
like = "I LIKE this"
dont_like = "I do not LIKE this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

In [ ]:
like = "I LIKE this"
dont_like = "I do NOT like this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

In [ ]:
like = "I like this"
dont_like = "I do NOT LIKE this"
print(sid.polarity_scores(like))
print(sid.polarity_scores(dont_like))

In [8]:
notbad = "not bad"
sid.polarity_scores(notbad)

{'neg': 0.0, 'neu': 0.26, 'pos': 0.74, 'compound': 0.431}

In [11]:
text = "I like you Tobias. I would also say, that I love you"
text_2 = "I like you @Tobias. I would also say, that I love you"
text_3 = "I #like you Tobias. I would also say, that I #love you"
text_4 = "I #like you @Tobias. I would also say, that I #love you"
print(sid.polarity_scores(text))
print(sid.polarity_scores(text_2))
print(sid.polarity_scores(text_3))
print(sid.polarity_scores(text_4))

{'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.7717}
{'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.7717}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


<h3>Flair</h3>

<p>I try to use the flair framework to do a sentiment analysis.</p>

In [2]:
flair_sentiment = flair.models.TextClassifier.load("en-sentiment")

2022-04-17 14:32:04,045 loading file /Users/tobias/.flair/models/sentiment-en-mix-distillbert_4.pt


In [23]:
def senti_score(n):
    s = flair.data.Sentence(n)
    print(s)
    flair_sentiment.predict(s)
    print(flair_sentiment.predict(s))
    total_sentiment = s.labels[0]
    print(total_sentiment)
    print(s.labels)
    assert total_sentiment.value in ['POSITIVE', 'NEGATIVE']
    sign = 1 if total_sentiment.value == 'POSITIVE' else -1
    score = total_sentiment.score
    return sign * score

In [45]:
print(s1, senti_score(s1))

Sentence: "Tesla is a great company . It has some weak ponints and sometimes they make big mistakes . Many People think it is the worst company . But I think it is ok ."   [− Tokens: 34]
None
POSITIVE (0.5541)
[POSITIVE (0.5541)]
Tesla is a great company. It has some weak ponints and sometimes they make big mistakes. Many People think it is the worst company. But I think it is ok. 0.5540907979011536


In [42]:
print(flair_sentiment.predict(flair.data.Sentence(s1)))

None


In [43]:
s = flair.data.Sentence(s1)
flair_sentiment.predict(s)
print(s.labels)

[NEGATIVE (0.9356)]


In [44]:
s1 = "Tesla is a great company. It has some weak ponints and sometimes they make big mistakes. Many People think it is the worst company. But I think it is ok."
s2 = "Tesla is a good company"
s3 = "Tesla is a company"
s4 = "Tesla is a bad company"

In [21]:
print(s1, senti_score(s1))
print(s2, senti_score(s2))
print(s3, senti_score(s3))
print(s4, senti_score(s4))

Tesla is a great company 0.9977720379829407
Tesla is a good company 0.9961444139480591
Tesla is a company 0.9995311498641968
Tesla is a bad company -0.9998192191123962


In [6]:
print(sid.polarity_scores(s1))
print(sid.polarity_scores(s2))
print(sid.polarity_scores(s3))
print(sid.polarity_scores(s4))

NameError: name 'sid' is not defined

In [68]:
flair_sentiment_all = flair.models.TextClassifier.load("de-offensive-language")

2022-03-24 15:04:39,506 https://nlp.informatik.hu-berlin.de/resources/models/de-offensive-language/germ-eval-2018-task-1-v0.8.pt not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpge1ck4wu


100%|█████████████████████████| 439837065/439837065 [03:57<00:00, 1848690.56B/s]

2022-03-24 15:08:37,638 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpge1ck4wu to cache at /Users/tobias/.flair/models/germ-eval-2018-task-1-v0.8.pt


2022-03-24 15:08:38,076 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpge1ck4wu
2022-03-24 15:08:38,096 loading file /Users/tobias/.flair/models/germ-eval-2018-task-1-v0.8.pt


Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/362 [00:00<?, ?B/s]

In [72]:
ds1 = "Tesla ist eine großartige Firma"
ds2 = "Tesla ist eine tolle Firma"
ds3 = "Tesla ist eine Firma"
ds4 = "Tesla ist eine böse Firma"

In [73]:
flair_sentiment_all.predict(ds1)

AttributeError: 'str' object has no attribute '_embeddings'

In [65]:
print(senti_score(ds1))
print(senti_score(ds2))
print(senti_score(ds3))
print(senti_score(ds4))

0.9990963935852051
0.9960978031158447
0.9998791217803955
0.999220609664917


In [ ]:
for each in range(15):
    print(sid.polarity_scores(df.iloc[each]["text"]))
    print(senti_score(df.iloc[each]["text"]))
    print(df.iloc[each]["text"])
    print(str(each)+"\n")
    

<h3>Stanza</h3>

In [5]:
#stanza.download("en")

2022-04-02 15:18:13 INFO: Downloading default packages for language: en (English)...


2022-04-02 15:19:20 INFO: Finished downloading models and saved to /Users/tobias/stanza_resources.


In [2]:
nlp = stanza.Pipeline(lang="en", processors="tokenize,sentiment")

2022-04-03 21:55:00 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2022-04-03 21:55:00 INFO: Use device: cpu
2022-04-03 21:55:00 INFO: Loading: tokenize
2022-04-03 21:55:00 INFO: Loading: sentiment
2022-04-03 21:55:01 INFO: Done loading processors!


In [3]:
s1 = "Tesla is a great company."
s2 = "Tesla is a good company."
s3 = "Tesla is just a company."
s4 = "Tesla is a bad company."
s5 = "Tesla might be a good company. It has some weak points and they are doing some bad studd. Overall I realy like the company. I think they are doing a great job and this is awesome."

In [4]:
doc = nlp("I hat that they banned Mox Opal. Tesla is a great company.")

In [5]:
text_list = [s1, s2, s3, s4, s5]

In [22]:
for i, sentence in enumerate(doc.sentences):
    print(i, sentence.sentiment)

0 2


In [23]:
doc = nlp(s1)

In [6]:
for each in range(len(text_list)):
    sentiment_value = []
    doc = nlp(text_list[each])
    for i, sentence in enumerate(doc.sentences):
        sentiment_value.append(sentence.sentiment)
    
    print(each, sentiment_value,(sum(sentiment_value)/len(sentiment_value)))

0 [2] 2.0
1 [2] 2.0
2 [1] 1.0
3 [0] 0.0
4 [2, 0, 1, 2] 1.25


In [30]:
#stanza.download("de")

2022-04-03 18:21:19 INFO: Downloading default packages for language: de (German)...


2022-04-03 18:22:13 INFO: Finished downloading models and saved to /Users/tobias/stanza_resources.


In [7]:
nlp_de = stanza.Pipeline(lang="de", processors="tokenize,sentiment")

2022-04-03 21:56:27 WARNING: Language de package default expects mwt, which has been added
2022-04-03 21:56:27 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| sentiment | sb10k   |

2022-04-03 21:56:27 INFO: Use device: cpu
2022-04-03 21:56:27 INFO: Loading: tokenize
2022-04-03 21:56:27 INFO: Loading: mwt
2022-04-03 21:56:27 INFO: Loading: sentiment
2022-04-03 21:56:27 INFO: Done loading processors!


In [9]:
de1 = "Tesla ist eine tolle Firma"
de2 = "Tesla ist die beste Firma" 
de3 = "Tesla ist eine scheiß Firma"
de4 = "Tesla ist keine besonders tolle Firma"
de5 = "Tesla ist eine ganz gute Firma. Sie hat ein paar schwächen und einige Aktionen sind scheiße. Dennoch ist es gut was sie erreicht hat. Sie hat etwas großartiges geschafft, was vorher noch keine andere Firma geschafft hat."
de6 = "Tesla ist die beste Firma auf der Welt. Sie ist so toll und absolut großartig!!!!"
de7 = "Ich bin am tollsten"
de8 = "Tesla ist die tollste Firma auf der Welt"
text_de_list = [de1, de2, de3, de4, de5, de6, de7, de8]

In [11]:
for each in range(len(text_de_list)):
    sentiment_value = []
    doc = nlp_de(text_de_list[each])
    for i, sentence in enumerate(doc.sentences):
        sentiment_value.append(sentence.sentiment)
    
    print(each, sentiment_value,(sum(sentiment_value)/len(sentiment_value)))

0 [1] 1.0
1 [1] 1.0
2 [1] 1.0
3 [1] 1.0
4 [1, 0, 1, 1] 0.75
5 [1, 2, 2] 1.6666666666666667
6 [2] 2.0
7 [1] 1.0


<h1>German-Sentiment</h1>

In [4]:
model = SentimentModel()

texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto.", "Tesla ist eine Firma", "Tesla ist die beste Firma auf der Welt!",
    "Tesla ist nicht besonders gut als Firma"]
       
result = model.predict_sentiment(texts)
print(result)

['negative', 'negative', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'positive', 'negative']


<h1>Load data from CSV file </h1>

In [2]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";")
df_wirecard_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Wirecard_DE/data.csv", sep=";")
df_manager_en = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_EN/data.csv", sep=";")
df_wirecard_en = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Wirecard_EN/data.csv", sep=";")

In [3]:
print("Manager_DE: " + str(len(df_manager_de)))
print("Wirecard_DE: " + str(len(df_wirecard_de)))
print("Manager_EN: " + str(len(df_manager_en)))
print("Wirecard_EN: " + str(len(df_wirecard_en)))

Manager_DE: 14308
Wirecard_DE: 165657
Manager_EN: 17801
Wirecard_EN: 203935


In [4]:
tweets_de = len(df_manager_de) + len(df_wirecard_de)
tweets_en = len(df_manager_en) + len(df_wirecard_en)

In [5]:
print("DE: " + str(tweets_de) + "\nEN: " + str(tweets_en))

DE: 179965
EN: 221736


In [6]:
df_de = pd.concat([df_manager_de, df_wirecard_de], ignore_index=True)

In [7]:
len(df_de)

179965

In [8]:
df_de = df_de.drop_duplicates(subset=["conversation_id"])

In [10]:
len(df_de)

150686

In [12]:
df_de = df_de.reset_index(drop=True)

In [13]:
df_de.iloc[71]

conversation_id                                  944209488080527360
author_id                                                2942451119
created_at                                2018-03-12 18:02:14+00:00
tweet_id                                         973257873844310016
lang                                                             de
retweet_count                                                     0
reply_count                                                       0
like_count                                                        0
quote_count                                                       0
source                                          Twitter for Android
tweet_type                                           ['replied_to']
referenced_tweet_id                          ['944209488080527360']
text                   @_MarkusBraun Jetzt einen schönen Wurstsalat
Name: 71, dtype: object

In [14]:
print(datetime.datetime.now())

for each in range(len(df_de)):
    #print(each) 17
    df_de.at[each,"tweet_type"] = re.findall(r"\w+", df_de.at[each,"tweet_type"])
    
print(datetime.datetime.now())

2022-03-26 10:16:36.062423
2022-03-26 10:16:37.889832


In [15]:
df_de.iloc[71]

conversation_id                                  944209488080527360
author_id                                                2942451119
created_at                                2018-03-12 18:02:14+00:00
tweet_id                                         973257873844310016
lang                                                             de
retweet_count                                                     0
reply_count                                                       0
like_count                                                        0
quote_count                                                       0
source                                          Twitter for Android
tweet_type                                             [replied_to]
referenced_tweet_id                          ['944209488080527360']
text                   @_MarkusBraun Jetzt einen schönen Wurstsalat
Name: 71, dtype: object

In [39]:
df_de.head(75)

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,[None],[None],#Wirecard-Chef Markus Braun pumpt irrwitzige S...
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,[None],[None],Jetzt günstig einsammeln und später absahnen? ...
2,701784983829143553,554491983,2016-02-22 15:05:53+00:00,701784983829143553,de,0,0,0,0,Facebook,[None],[None],Morgen startet die neue Reihe R²-JazzCube! Vol...
3,701446667170615296,3365669884,2016-02-21 16:41:32+00:00,701446667170615296,de,0,0,1,0,Twitter Web Client,[None],[None],die Y-Achse des #prusai3 nimmt Formen an :-) ...
4,700597812556386304,4858383167,2016-02-19 08:28:29+00:00,700597812556386304,de,2,0,0,0,Twitter for iPhone,[retweeted],['700578193334411264'],RT @aktionaer: #Wirecard: Was führt der Vorsta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,945973335258787840,917052505430863873,2018-03-14 08:36:34+00:00,973840298274164738,de,0,0,0,0,Twitter for Android,[replied_to],['945973335258787840'],"@_MarkusBraun Man sollte sich überlegen, einen..."
71,944209488080527360,2942451119,2018-03-12 18:02:14+00:00,973257873844310016,de,0,0,0,0,Twitter for Android,[replied_to],['944209488080527360'],@_MarkusBraun Jetzt einen schönen Wurstsalat
72,973195874888290304,751899999752036354,2018-03-12 14:16:41+00:00,973201112219217921,de,0,1,0,0,Twitter Web Client,[replied_to],['973195874888290304'],@Ademm66Mr @wirecard @bankenverband @_MarkusBr...
73,984792434533191680,17771831,2018-04-13 13:56:27+00:00,984792434533191680,de,0,0,0,0,IFTTT,[None],[None],RT @BlarryOfficial: @bvbflo09 @threalNox @seba...


<h2>Preprocessing Data</h2>

In [37]:
for each in range(len(df_manager_de)):
    df_manager_de.at[each,"tweet_type"] = re.findall(r"\w+", df_manager_de.at[each, "tweet_type"])
    #print(each)

In [22]:
counter = 0
multiple_tweet_types = []
for each in range(len(df_de)):
    if len(df_de["tweet_type"][each]) > 1:
        multiple_tweet_types.append(each)
        counter += 1
print(counter)
print(multiple_tweet_types)


189
[1854, 3491, 4568, 5346, 5481, 5866, 5910, 5911, 5912, 5934, 5935, 5936, 5937, 5938, 5969, 5972, 5973, 5975, 5976, 5998, 5999, 6001, 6002, 6003, 6004, 6005, 6006, 6018, 6020, 6029, 6030, 6031, 6032, 6033, 6034, 6035, 6041, 6051, 6052, 6053, 6054, 6055, 6056, 6057, 6058, 6059, 6060, 6061, 6062, 6063, 6064, 6065, 6066, 6079, 6080, 6081, 6082, 6083, 6084, 6085, 6086, 6087, 6091, 6095, 6096, 6097, 6099, 6100, 6101, 6102, 6103, 6104, 6105, 6106, 6107, 6108, 6109, 6116, 6118, 6119, 6124, 6125, 6127, 6128, 6130, 6131, 6147, 6150, 6153, 6155, 6156, 6158, 6160, 6162, 6172, 6173, 6174, 6175, 6176, 6178, 6179, 6181, 6182, 6184, 6186, 6189, 6190, 6191, 6194, 6196, 6197, 6198, 6199, 6200, 6204, 6206, 6207, 6208, 6210, 6212, 6213, 6340, 9629, 9675, 16145, 18216, 18939, 29871, 38798, 39281, 44789, 46703, 59083, 62656, 63405, 65430, 67090, 70141, 70196, 70201, 70574, 71364, 76438, 76439, 78110, 78456, 79410, 80123, 80476, 80711, 81357, 81958, 87878, 91141, 91548, 92952, 95761, 96130, 97522, 97894,

In [46]:
tweet_list = df_de.iloc[71]["tweet_type"]

In [47]:
df_de[df_de["tweet_type"] == tweet_list]

ValueError: ('Lengths must match to compare', (150686,), (1,))

In [1]:
from flair.embeddings import WordEmbeddings
from flair.data import Sentence

# init embedding
glove_embedding = WordEmbeddings('glove')

2022-04-05 22:00:29,776 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmp0s7vdsjv


100%|████████████████████████| 160000128/160000128 [00:15<00:00, 10463324.56B/s]

2022-04-05 22:00:45,184 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmp0s7vdsjv to cache at /Users/tobias/.flair/embeddings/glove.gensim.vectors.npy


2022-04-05 22:00:45,241 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmp0s7vdsjv
2022-04-05 22:00:45,344 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpr99exaoa


100%|██████████████████████████| 21494764/21494764 [00:01<00:00, 11938174.10B/s]

2022-04-05 22:00:47,237 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpr99exaoa to cache at /Users/tobias/.flair/embeddings/glove.gensim


2022-04-05 22:00:47,245 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpr99exaoa


In [2]:
# create sentence.
sentence = Sentence('The grass is green .')

# embed a sentence using glove.
glove_embedding.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  

In [3]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings

# init standard GloVe embedding
glove_embedding = WordEmbeddings('glove')

# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

2022-04-05 22:09:50,450 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpubqr77v9


100%|██████████████████████████| 73034624/73034624 [00:06<00:00, 11105076.91B/s]

2022-04-05 22:09:57,122 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpubqr77v9 to cache at /Users/tobias/.flair/embeddings/news-forward-0.4.1.pt


2022-04-05 22:09:57,148 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpubqr77v9
2022-04-05 22:09:57,495 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpa2m43ay0


100%|██████████████████████████| 73034575/73034575 [00:05<00:00, 12236186.04B/s]

2022-04-05 22:10:03,580 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpa2m43ay0 to cache at /Users/tobias/.flair/embeddings/news-backward-0.4.1.pt
2022-04-05 22:10:03,610 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpa2m43ay0


In [4]:
from flair.embeddings import StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        glove_embedding,
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [5]:
sentence = Sentence('The grass is green .')

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-0.0382, -0.2449,  0.7281,  ..., -0.0065, -0.0053,  0.0090])
Token: 2 grass
tensor([-0.8135,  0.9404, -0.2405,  ...,  0.0354, -0.0255, -0.0143])
Token: 3 is
tensor([-5.4264e-01,  4.1476e-01,  1.0322e+00,  ..., -5.3691e-04,
        -9.6750e-03, -2.7541e-02])
Token: 4 green
tensor([-0.6791,  0.3491, -0.2398,  ..., -0.0007, -0.1333,  0.0161])
Token: 5 .
tensor([-0.3398,  0.2094,  0.4635,  ...,  0.0005, -0.0177,  0.0032])


In [1]:
from flair.data import Corpus
from flair.datasets import TREC_6

In [2]:
corpus: Corpus = TREC_6()

2022-04-06 21:45:32,800 https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpet36o6uk


100%|████████████████████████████████| 335858/335858 [00:00<00:00, 865448.52B/s]

2022-04-06 21:45:33,531 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpet36o6uk to cache at /Users/tobias/.flair/datasets/trec_6/original/train_5500.label
2022-04-06 21:45:33,534 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmpet36o6uk


2022-04-06 21:45:33,865 https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label not found in cache, downloading to /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmplvot00lj


100%|██████████████████████████████████| 23354/23354 [00:00<00:00, 229140.75B/s]

2022-04-06 21:45:34,297 copying /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmplvot00lj to cache at /Users/tobias/.flair/datasets/trec_6/original/TREC_10.label
2022-04-06 21:45:34,299 removing temp file /var/folders/3r/0dwb2f6j0f39f4r2j9yyjf180000gn/T/tmplvot00lj
2022-04-06 21:45:34,317 Reading data from /Users/tobias/.flair/datasets/trec_6
2022-04-06 21:45:34,317 Train: /Users/tobias/.flair/datasets/trec_6/train.txt
2022-04-06 21:45:34,318 Dev: None
2022-04-06 21:45:34,318 Test: /Users/tobias/.flair/datasets/trec_6/test.txt


2022-04-06 21:45:34,888 Initialized corpus /Users/tobias/.flair/datasets/trec_6 (label type name is 'question_class')


In [1]:
>>> from transformers import pipeline

# Allocate a pipeline for sentiment-analysis
>>> classifier = pipeline('sentiment-analysis')
>>> classifier('We are very happy to introduce pipeline to the transformers repository.')
[{'label': 'POSITIVE', 'score': 0.9996980428695679}]

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9996980428695679}]

In [3]:
classifier("Tesla is a great company")

[{'label': 'POSITIVE', 'score': 0.9998736381530762}]

In [4]:
classifier("Tesla is a bad company")

[{'label': 'NEGATIVE', 'score': 0.9997842907905579}]

In [5]:
classifier("Tesla is a company")

[{'label': 'POSITIVE', 'score': 0.9983876943588257}]

In [6]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [9]:
classifier("Tesla is a great company")

[{'label': '5 stars', 'score': 0.805143415927887}]

In [10]:
classifier("Tesla is a bad company")

[{'label': '1 star', 'score': 0.791061520576477}]

In [11]:
classifier("Tesla is a company")

[{'label': '5 stars', 'score': 0.45207613706588745}]

[{'label': '5 stars', 'score': 0.40043339133262634}]

<h1>Test Load Data </h1>

In [3]:
df_WRCDF_1 = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/WRCDF_EN/data.csv", sep=";")
df_WRCDF_2 = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data.csv", sep=";")

In [8]:
df_s = [df_WRCDF_1, df_WRCDF_2]

In [102]:
df = pd.concat(df_s, ignore_index=True)

In [103]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703293739700260865,61661638,2016-02-26 19:01:08+00:00,703293739700260865,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...
1,702932443154817024,61661638,2016-02-25 19:05:28+00:00,702932443154817024,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Stock Rating Reaffirmed...
2,702047773114281984,926793032,2016-02-23 08:30:07+00:00,702047773114281984,en,0,0,0,0,First To Markets,[None],[None],Wirecard AG Reiterated Rating by DZ Bank AG (...
3,702046074899402752,68559732,2016-02-23 08:23:22+00:00,702046074899402752,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv's buy rating reiterated at DZ ...
4,702045692580204544,2920258887,2016-02-23 08:21:51+00:00,702045692580204544,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...


In [96]:
print("df: ", len(df))
print("df_WRCDF_1: ", len(df_WRCDF_1))
print("df_WRCDF_2: ", len(df_WRCDF_2))

df:  1887
df_WRCDF_1:  627
df_WRCDF_2:  1260


In [63]:
len(df_WRCDF_1)+len(df_WRCDF_2)

1887

In [95]:
len(df.drop_duplicates())

1261

In [104]:
df.at[0, 'retweet_count'] = 100

In [105]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703293739700260865,61661638,2016-02-26 19:01:08+00:00,703293739700260865,en,100,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...
1,702932443154817024,61661638,2016-02-25 19:05:28+00:00,702932443154817024,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Stock Rating Reaffirmed...
2,702047773114281984,926793032,2016-02-23 08:30:07+00:00,702047773114281984,en,0,0,0,0,First To Markets,[None],[None],Wirecard AG Reiterated Rating by DZ Bank AG (...
3,702046074899402752,68559732,2016-02-23 08:23:22+00:00,702046074899402752,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv's buy rating reiterated at DZ ...
4,702045692580204544,2920258887,2016-02-23 08:21:51+00:00,702045692580204544,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...


In [65]:
len(df_WRCDF_2)-len(df_WRCDF_1)

633

In [81]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,tweet_count
0,703293739700260865,61661638,2016-02-26 19:01:08+00:00,703293739700260865,en,100,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...,100.0
1,702932443154817024,61661638,2016-02-25 19:05:28+00:00,702932443154817024,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Stock Rating Reaffirmed...,NaN
2,702047773114281984,926793032,2016-02-23 08:30:07+00:00,702047773114281984,en,0,0,0,0,First To Markets,[None],[None],Wirecard AG Reiterated Rating by DZ Bank AG (...,NaN
3,702046074899402752,68559732,2016-02-23 08:23:22+00:00,702046074899402752,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv's buy rating reiterated at DZ ...,NaN
4,702045692580204544,2920258887,2016-02-23 08:21:51+00:00,702045692580204544,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...,NaN


In [38]:
number = 703293739700260865
print(number, "number type: ", type(number))

703293739700260865 number type:  <class 'int'>


In [109]:
df.loc[df['conversation_id'] == 703293739700260865]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
0,703293739700260865,61661638,2016-02-26 19:01:08+00:00,703293739700260865,en,100,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Earns Buy Rating from D...


In [110]:
df.loc[df['conversation_id'] == 702932443154817024]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
1,702932443154817024,61661638,2016-02-25 19:05:28+00:00,702932443154817024,en,0,0,0,0,IFTTT,[None],[None],Wirecard Ag Npv $WRCDF Stock Rating Reaffirmed...


In [108]:
df = df.drop_duplicates(subset=['conversation_id'])